In [ ]:
import cv2
import dlib
import imutils
from imutils import face_utils
import numpy as np
from scipy.spatial import distance as dist
import time
import winsound

# Play alarm sound when attention is drowsy for 3 seconds
def play_alarm():
    #time.sleep(3)
    frequency = 3500  # Set Frequency To 2500 Hertz
    duration = 5000  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)
   

# Calculate the eye aspect ratio
def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # Compute the euclidean distance between the horizontal eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # Return the eye aspect ratio
    return ear

# Initialize dlib's face detector and create a dictionary to map facial landmark indexes to face parts
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
face_parts = face_utils.FACIAL_LANDMARKS_IDXS
(left_eye_start, left_eye_end) = face_parts["left_eye"]
(right_eye_start, right_eye_end) = face_parts["right_eye"]

# Start the video stream and wait for the camera to warm up
vs = cv2.VideoCapture(0)
time.sleep(2.0)

# Initialize variables for calculating the attention level
drowsy_counter = 0
drowsy_threshold = 15
focused_counter = 0
focused_threshold = 5
unfocused_counter=0;
head_direction_threshold = 5
head_direction_degrees = 0
max_attention_level = "unfocused"
max_attention_time = time.time()

# Initialize blink detector
blink_frames = 0
blink_threshold = 3
eyes_closed = False

# Loop over frames from the video stream
while True:
    # Read the next frame from the video stream and resize it
    _, frame = vs.read()
    frame = imutils.resize(frame, width=450)

    # Convert the frame to grayscale and detect faces
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    # Loop over the face detections
    for rect in rects:
        # Determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # Extract the left and right eye coordinates and compute the eye aspect ratio for each eye
        left_eye = shape[left_eye_start:left_eye_end]
        right_eye = shape[right_eye_start:right_eye_end]
        left_eye_ear = eye_aspect_ratio(left_eye)
        right_eye_ear = eye_aspect_ratio(right_eye)

        # Compute the average eye aspect ratio
        avg_ear = (left_eye_ear + right_eye_ear) / 2.0
        
        
        # Check if the head is directed towards the screen
        # Calculate the angle between the eye line and the horizontal line
        left_eye_center = np.mean(left_eye, axis=0).astype(int)
        right_eye_center = np.mean(right_eye, axis=0).astype(int)
        dx = right_eye_center[0] - left_eye_center[0]
        dy = right_eye_center[1] - left_eye_center[1]
        angle = np.degrees(np.arctan2(dy, dx))
        if angle < 0:
            angle += 360
        head_direction_degrees = angle
#         print(head_direction_degrees)
        if head_direction_degrees > 185 or head_direction_degrees < 170:
            unfocused_counter+=1
            drowsy_counter = 0
            focused_counter=0
        else:
            unfocused_counter=0

        # Check if the average eye aspect ratio is below the drowsy threshold
        if avg_ear < 0.25:
            drowsy_counter += 1
            focused_counter = 0
        else:
            drowsy_counter = 0
            focused_counter += 1

        # Check if the attention level is drowsy or unfocused for more than 3 seconds
        if drowsy_counter >= drowsy_threshold:
            max_attention_level = "drowsy"
            if drowsy_counter == drowsy_threshold:
                alarm_start_time = time.time()
            if time.time() - alarm_start_time >= 3 and not eyes_closed:
                play_alarm()
                eyes_closed = True
                drowsy_counter = 0
        elif focused_counter >= focused_threshold:
            max_attention_level = "focused"
            eyes_closed = False
            
            
        elif unfocused_counter >= head_direction_threshold:
                max_attention_level = "unfocused"
                unfocused_counter=0
                eyes_closed = False
	  
 # Update the max attention level
    if time.time() - max_attention_time >= 10:
        print(f"Max attention level for the past 10 Seconds : {max_attention_level}")
        max_attention_level = "unfocused"
        max_attention_time = time.time()

    # Display the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # If the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# Cleanup
cv2.destroyAllWindows()
#vs.stop()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2023-04-20 10:30:44,732] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "E:\New folder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "E:\New folder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "E:\New folder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "E:\New folder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "E:\New folder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "E:\New folder\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Dell\AppData\Local\Temp\ipykernel_8500\2104775771.py", line 78, in login
    re